## Chinook Database

The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

* contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. 
* This information is contained in eleven tables.

<img src="tables.png">

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("chinook.db")

## 3 tables: invoice_line, track, media_type

In [3]:
#Visualizing a few tables in the database
invoice_line = "SELECT * FROM invoice_line"
track = "SELECT * FROM track"
media_type = "SELECT * FROM media_type"

In [4]:
print("invoice_line table")
pd.read_sql_query(invoice_line, conn).head()

invoice_line table


,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,1,1,1158,0.99,1
1,2,1,1159,0.99,1
2,3,1,1160,0.99,1
3,4,1,1161,0.99,1
4,5,1,1162,0.99,1


In [5]:
print("track table")
pd.read_sql_query(track, conn).head()

track table


,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [6]:
print("media_type table")
pd.read_sql_query(media_type, conn).head()

media_type table


,media_type_id,name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


## Invoce_id = 4 

Table contains:
* id from track, **track_id**
* name of the track, **track_name**
* Name of media type of the track, **track_type**
* Price paid by customer for the track, **unit_price**
* Quantity of the track purchased, **quantity**

In [7]:
invoice_id_4 = "SELECT il.track_id, tr.name track_name, me.name track_type, il.unit_price, il.quantity FROM invoice_line il INNER JOIN(SELECT * FROM track) tr ON tr.track_id = il.track_id INNER JOIN(SELECT * FROM media_type) me ON me.media_type_id = tr.media_type_id WHERE invoice_id = 4"
pd.read_sql_query(invoice_id_4, conn)

,track_id,track_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",Protected AAC audio file,0.99,1
1,2560,Violent Pornography,MPEG audio file,0.99,1
2,3336,War Pigs,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,MPEG audio file,0.99,1
4,1872,Attitude,MPEG audio file,0.99,1
5,748,Dealer,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
7,2514,Spoonman,MPEG audio file,0.99,1


## Adding column from artist table
In order to add join artist table, another table needs to be joined: album, which has information that helps connect artist table with our previous table
* invoice_line > track > media_type > album > artist

In [8]:
#querying table
invoiceid_4_artist = "SELECT il.track_id, tr.name track_name, ar.name artist_name, me.name track_type, il.unit_price, il.quantity FROM invoice_line il INNER JOIN(SELECT * FROM track) tr ON tr.track_id = il.track_id INNER JOIN(SELECT * FROM media_type) me ON me.media_type_id = tr.media_type_id INNER JOIN(SELECT * FROM album) al ON al.album_id = tr.album_id INNER JOIN(SELECT * FROM artist) ar ON ar.artist_id = al.artist_id WHERE invoice_id = 4"
pd.read_sql_query(invoiceid_4_artist, conn)

,track_id,track_name,artist_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",The King's Singers,Protected AAC audio file,0.99,1
1,2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
2,3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
4,1872,Attitude,Metallica,MPEG audio file,0.99,1
5,748,Dealer,Deep Purple,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
7,2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


**New table to be used: artist**

In [9]:
#Table connects to track track through artist_id
artist = "SELECT * FROM artist LIMIT 5"
pd.read_sql_query(artist, conn)

,artist_id,name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


## Top 5 albums by # tracks purchased
The queary returns top 5 albums, calculated by the number of times a track from that album has been purchased.
* album, title of the album
* artist, artist who produced the album
* tracks_purchased the total number of tracks purchased from that album.

Query should list the albums from most tracks purchased to least.

In [10]:
top_album_by_trackspurchased = "SELECT album, artist, COUNT(*) tracks_purchased FROM invoice_line il INNER JOIN(SELECT t.track_id, al.title album, ar.name artist FROM track t INNER JOIN album al ON al.album_id = t.album_id INNER JOIN artist ar ON ar.artist_id = al.artist_id) ta ON ta.track_id = il.track_id GROUP BY 1 ORDER BY 3 DESC LIMIT 5"
pd.read_sql_query(top_album_by_trackspurchased, conn)

,album,artist,tracks_purchased
0,Are You Experienced?,Jimi Hendrix,187
1,Faceless,Godsmack,96
2,Mezmerize,System Of A Down,93
3,Get Born,JET,90
4,The Doors,The Doors,83


**New table: employee**

In [11]:
employee = "SELECT * FROM employee LIMIT 5"
pd.read_sql_query(employee, conn)

,employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


We can see that in our employee table, where there is a reports_to column that has a relation to the employee_id column within the same table.

The reports_to column identifies each employee's supervisor

## Report Employee - Supervisor

Write a query that returns information about each employee and their supervisor. Your query should return the following columns, in order:
* **employee_name** - containing the first_name and last_name columns separated by a space, eg Luke Skywalker
* **employee_title** - the title of that employee
* **supervisor_name** - the first and last name of the person the employee reports to, in the same format as employee_name
* **supervisor_title** - the title of the person the employee reports to.

The report should include employees even if they do not report to another employee.
The report should be sorted alphabetically by the employee_name column.

In [12]:
employee_supervisor = "SELECT e1.first_name || ' ' || e1.last_name employee_name, e1.title employee_title, e2.first_name || ' ' || e2.last_name supervisor_name, e2.title supervisor_title FROM employee e1 LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id ORDER BY 1"
pd.read_sql_query(employee_supervisor, conn)

,employee_name,employee_title,supervisor_name,supervisor_title
0,Andrew Adams,General Manager,None,None
1,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Laura Callahan,IT Staff,Michael Mitchell,IT Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael Mitchell,IT Manager,Andrew Adams,General Manager
5,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


**New table: customer**

In [13]:
customer = "SELECT * FROM customer LIMIT 5"
pd.read_sql_query(customer, conn)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


## Using LIKE

You have just returned from lunch to see another phone message on your desk: "Call Belle." 

Write a query that finds the contact details of a customer with a first_name containing Belle from the database.

Your query should include the following columns, in order:
* first_name
* last_name
* phone

In [14]:
call_belle = "SELECT first_name, last_name, phone FROM customer WHERE first_name LIKE '%Belle%'"
pd.read_sql_query(call_belle, conn)

,first_name,last_name,phone
0,Isabelle,Mercier,+33 03 80 73 66 99


**New table: media_type & Invoice**

In [15]:
media_type = "SELECT * FROM media_type"
pd.read_sql_query(media_type, conn)

,media_type_id,name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [16]:
invoice = "SELECT * FROM invoice LIMIT 5"
pd.read_sql_query(invoice, conn)

,invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
1,2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.90
2,3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
3,4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
4,5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


## USING CASE and LIKE 

Write a query that summarizes the purchases of each customer. For the purposes of this exercise, we do not have any two customers with the same name.

Your query should include the following columns, in order:

* **customer_name** - containing the first_name and last_name columns separated by a space, eg Luke Skywalker.
* **number_of_purchases**, counting the number of purchases made by each customer.
* **total_spent** - the total sum of money spent by each customer.
* **customer_category** - a column that categorizes the customer based on their total purchases. The column should contain the following values:
     1. **small spender** - If the customer's total purchases are less than 40dollars.
     2. **big spender** - If the customer's total purchases are greater than 100dollars.
     3. **regular** - If the customer's total purchases are between 40dollars and 100dollars (inclusive).

Order your results by the customer_name column.

In [17]:
customer_purchase = "SELECT c.first_name || ' ' || c.last_name customer_name, COUNT(i.invoice_id) number_of_purchases, SUM(i.total) total_spent, CASE WHEN SUM(i.total) < 40 THEN 'small spender' WHEN SUM(i.total) > 100 THEN 'big spender' ELSE 'regular' END AS customer_category FROM invoice i INNER JOIN customer c ON i.customer_id = c.customer_id GROUP BY 1 ORDER BY 1"
pd.read_sql_query(customer_purchase, conn).head()

,customer_name,number_of_purchases,total_spent,customer_category
0,Aaron Mitchell,8,70.29,regular
1,Alexandre Rocha,10,69.30,regular
2,Astrid Gruber,9,69.30,regular
3,Bjørn Hansen,9,72.27,regular
4,Camille Bernard,9,79.20,regular
